In [0]:
# 01_bronze_ingestion

**Purpose:**  
Generate realistic historical shipment data and store it as a Bronze Delta table.

**Layer:** Bronze  
**Output Table:** logistics_bronze.shipments_raw

In [0]:
%python
from datetime import datetime, timedelta
import random

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS logistics_bronze;

In [0]:
cities = [
    ("Mumbai", "Maharashtra"),
    ("Delhi", "Delhi"),
    ("Bengaluru", "Karnataka"),
    ("Chennai", "Tamil Nadu"),
    ("Hyderabad", "Telangana"),
    ("Pune", "Maharashtra"),
    ("Ahmedabad", "Gujarat"),
    ("Jaipur", "Rajasthan"),
    ("Kolkata", "West Bengal"),
    ("Indore", "Madhya Pradesh")
]

carriers = [
    "BlueDart",
    "Delhivery",
    "EcomExpress",
    "DTDC",
    "XpressBees"
]

shipping_modes = ["Road", "Air"]

In [0]:
def generate_dates():
    order_date = datetime(2023, 1, 1) + timedelta(days=random.randint(0, 364))

    expected_days = random.choice([2, 3, 4, 5, 6])
    expected_delivery = order_date + timedelta(days=expected_days)

    delay = random.choices(
        population=[-1, 0, 1, 2, 3],
        weights=[10, 55, 20, 10, 5]
    )[0]

    actual_delivery = expected_delivery + timedelta(days=delay)

    return (
        order_date.date(),
        expected_delivery.date(),
        actual_delivery.date()
    )

In [0]:
def generate_shipment(shipment_id):
    origin_city, origin_state = random.choice(cities)
    destination_city, destination_state = random.choice(cities)

    while destination_city == origin_city:
        destination_city, destination_state = random.choice(cities)

    order_date, expected_date, actual_date = generate_dates()

    distance_km = random.randint(100, 2200)
    weight = round(random.uniform(1, 50), 2)

    carrier = random.choice(carriers)
    shipping_mode = random.choices(
        shipping_modes,
        weights=[70, 30]
    )[0]

    return (
        f"SHP{shipment_id:06d}",
        order_date,
        expected_date,
        actual_date,
        origin_city,
        destination_city,
        origin_state,
        destination_state,
        distance_km,
        carrier,
        shipping_mode,
        weight,
        "Delivered"
    )

In [0]:
data = [generate_shipment(i) for i in range(1, 10001)]

In [0]:
columns = [
    "shipment_id",
    "order_date",
    "expected_delivery_date",
    "actual_delivery_date",
    "origin_city",
    "destination_city",
    "origin_state",
    "destination_state",
    "distance_km",
    "carrier",
    "shipping_mode",
    "shipment_weight_kg",
    "delivery_status"
]

shipments_df = spark.createDataFrame(data, columns)

In [0]:
shipments_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("logistics_bronze.shipments_raw")

In [0]:
from pyspark.sql.functions import col

In [0]:
spark.sql("""
SELECT COUNT(*) AS total_shipments
FROM logistics_bronze.shipments_raw
""").show()

+---------------+
|total_shipments|
+---------------+
|          10000|
+---------------+



In [0]:
spark.sql("""
SELECT
  actual_delivery_date > expected_delivery_date AS is_late,
  COUNT(*) AS count
FROM logistics_bronze.shipments_raw
GROUP BY is_late
""").show()

+-------+-----+
|is_late|count|
+-------+-----+
|   true| 3543|
|  false| 6457|
+-------+-----+

